In [8]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [9]:
INPUT_SIZE = 256
image_directory = r'E:/Project/2024 Project/BrainTumor_Lam/datasets/'

no_tumor_images = os.listdir(image_directory + 'sort_crop_no/')
yes_tumor_images = os.listdir(image_directory + 'sort_crop_yes/')
dataset = []
labels = []


In [10]:
for image_name in no_tumor_images:
    if image_name.endswith('.jpg'):
        image = cv2.imread(image_directory + 'sort_crop_no/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        labels.append(0)

for image_name in yes_tumor_images:
    if image_name.endswith('.jpg'):
        image = cv2.imread(image_directory + 'sort_crop_yes/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        labels.append(1)

In [11]:
dataset = np.array(dataset)
labels = np.array(labels)

x_train, x_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, random_state=42)

x_train = torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

x_train = x_train.permute(0, 3, 1, 2) / 255.0  # Permute dimensions for PyTorch (N, C, H, W)
x_test = x_test.permute(0, 3, 1, 2) / 255.0

train_data = torch.utils.data.TensorDataset(x_train, y_train)
test_data = torch.utils.data.TensorDataset(x_test, y_test)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)


In [12]:
class BrainTumorNet(nn.Module):
    def __init__(self):
        super(BrainTumorNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * (INPUT_SIZE // 8) * (INPUT_SIZE // 8), 64)
        self.fc2 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.3)  # Increased dropout rate to 0.3 for regularization

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 64 * (INPUT_SIZE // 8) * (INPUT_SIZE // 8))
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        return x

In [13]:
model = BrainTumorNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [14]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True, min_lr=1e-6)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device, dtype=torch.float32)
        
        optimizer.zero_grad()
        
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device, dtype=torch.float32)
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f"Validation Loss: {val_loss/len(test_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")
    
    # Adjust learning rate
    scheduler.step(val_loss)

save_path = 'E:/Project/2024 Project/BrainTumor_Lam/models/brain_tumor_pytorch_v1.pth'
# Save the model
torch.save(model.state_dict(), save_path)


Epoch [1/10], Loss: 0.6753
Validation Loss: 0.6172, Accuracy: 66.13%
Epoch [2/10], Loss: 0.5864
Validation Loss: 0.5166, Accuracy: 75.87%
Epoch [3/10], Loss: 0.4522
Validation Loss: 0.3712, Accuracy: 83.53%
Epoch [4/10], Loss: 0.3217
Validation Loss: 0.3064, Accuracy: 87.47%
Epoch [5/10], Loss: 0.2333
Validation Loss: 0.2318, Accuracy: 89.33%
Epoch [6/10], Loss: 0.1672
Validation Loss: 0.2279, Accuracy: 91.65%
Epoch [7/10], Loss: 0.1361
Validation Loss: 0.2505, Accuracy: 89.79%
Epoch [8/10], Loss: 0.0940
Validation Loss: 0.2334, Accuracy: 92.34%
Epoch [9/10], Loss: 0.0658
Validation Loss: 0.2611, Accuracy: 91.42%
Epoch [10/10], Loss: 0.0521
Validation Loss: 0.2813, Accuracy: 92.81%


In [16]:
model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs).squeeze()
        predicted = (outputs > 0.5).cpu().numpy().astype(int)
        y_pred.extend(predicted)
        y_true.extend(labels.numpy())

f1 = f1_score(y_true, y_pred)
print(f"F1 Score: {f1}")

F1 Score: 0.9238329238329238
